In [13]:
# this is a little trick to make sure the the notebook takes up most of the screen:
from IPython.display import HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Recommendation to leave the logging config like this, otherwise you'll be flooded with unnecessary info
import logging
logging.basicConfig(level=logging.WARNING, format='%(levelname)s:%(message)s')  


# Recommendation: logging config like this, otherwise you'll be flooded with unnecessary information
import logging
logging.basicConfig(level=logging.ERROR)

import sys
sys.path.append('../')

In [14]:
# import all modeling concepts
from crestdsl.model import *

# import the simulator
from crestdsl.simulation import Simulator

# import the plotting libraries that can visualise the CREST systems
from crestdsl.ui import elk

# we will create tests for each Entity
import unittest

class TestClass(unittest.TestCase):
    @classmethod
    def runall(cls):
        tests = unittest.TestLoader().loadTestsFromTestCase(cls)
        return unittest.TextTestRunner().run(tests)
    


In [15]:
class Resources(object):
    electricity = Resource("Watt", REAL)
    switch = Resource("switch", ["on", "off"])
    pourcent = Resource("%", REAL) 
    light = Resource("Lumen", INTEGER)
    time = Resource("minutes", REAL)
    water = Resource("litre", REAL)
    celsius = Resource("Celsius", REAL)
    boolean = Resource("bool", BOOL)
    presence = Resource("presence", ["detected", "no presence"])
    onOffAuto = Resource("onOffAutoSwitch", ["on", "off", "auto"])
    integer = Resource("integer", INTEGER)
    weight = Resource("kg", REAL)
    lenght = Resource("m", REAL)
    area = Resource("m²", REAL)

In [16]:
class ElectricalDevice(object):
    electricity_in = Input(Resources.electricity, value=0)
    req_electricity_out = Output(Resources.electricity, value=0)
    
class WaterDevice(object):
    water_in = Input(Resources.water, value=0)
    req_water_out = Output(Resources.water, value=0)

In [27]:
# TV modelised in a connected house, can be turned on only if the dishwasher is off, and the DW can't be lauched if TV is on
# Electricity consumption is take in account and is adaptable as all the electrical devices 

@dependency(source="req_electricity_out", target="switch_TV")
class TV(Entity, ElectricalDevice):
    
    """ - - - - - - - PORTS - - - - - - - - - - """
    #elec_In=Input(Resources.electricity, 0)
    
    switch_TV = Input(Resources.switch, value="off")
    
    switch_NoiseDW = Output(resource=Resources.switch, value="off") 
    
    cons_TV = Local(Resources.electricity, 328)
    
    def __init__(self, consumption=328):
        self.cons_TV.value = consumption

    """ - - - - - - - STATES & TRANSITIONS - - - - - - - - - - """
    off = current = State()
    on = State()
    
    off_to_on = Transition(off, on, guard=(lambda self: self.switch_TV.value == "on"))
    on_to_off = Transition(on, off, guard=(lambda self: self.electricity_in.value < self.cons_TV.value or self.switch_TV.value == "off"))
    
    """ - - - - - - - UPDATES - - - - - - - - - - """
    
    @update(state=off, target=switch_NoiseDW)
    def update_switch_NoiseDW_out(self, dt):
        # no electricity
        return "off"
    
    @update(state=on, target=switch_NoiseDW)
    def update_switch_NoiseDW_out(self, dt):
        # no electricity
        return "on"
    
    @update(state=off, target="req_electricity_out")
    def set_req_elec(self, dt):
        if self.switch_TV.value == "on":
            return self.cons_TV.value
        else:        
            return 0
        
    @update(state=on, target="req_electricity_out")
    def set_req_elec(self, dt):
        return self.cons_TV.value

# create an instance!
elk.plot(TV())

In [26]:
#lights that gets two states, on and off. For lights that can be brockened (more realistic ones maybe) you can go in the CrestCorp file "lamps"

@dependency(source="req_electricity_out", target="switch_in")
class Lights(Entity, ElectricalDevice):
    
    """ - - - - - - - PORTS - - - - - - - - - - """
    switch_in = Input(Resources.switch, value = "off")
    light = Output(resource=Resources.light, value=300)
    
    cons_Lights = Local(Resources.electricity, 50)
    
    def __init__(self, consumption=50):
        self.cons_Lights.value = consumption
    
    """ - - - - - - - INFLUENCES - - - - - - - - - - """
    
    """ - - - - - - - STATES & TRANSITIONS - - - - - - - - - - """
    on = State()
    off = current = State()
    
    off_to_on = Transition(source=off, target=on, guard=(lambda self: self.switch_in.value == "on"))
    on_to_off = Transition(source=on, target=off, guard=(lambda self: self.switch_in.value == "off"))
    
    """ - - - - - - - UPDATES - - - - - - - - - - """

    @update(state=on, target=light)
    def set_light_on(self, dt=0):
        
        if self.electricity_in.value >= self.cons_Lights.value :
        
            return 200 #random value
        else:
            return 0

    @update(state=off, target=light)
    def set_light_off(self, dt=0):
        return 0
    
    @update(state=off, target="req_electricity_out")
    def set_light_req_elec(self, dt):
        return 0
        
    @update(state=on, target="req_electricity_out")
    def set_light_req_elec(self, dt):
        
        return self.cons_Lights.value
    
# create an instance!
elk.plot(Lights())

In [19]:
#Taken form CrestCorp file, handle the electricity consumption and send the energy from the "electric part" to all the connected devices

class Splitter(Entity, ElectricalDevice):
    normal = current = State()
    no_power = State()
    
    
    req_sum = Local(Resources.electricity, value=0)
    req_sum_to_req = Influence(source=req_sum, target="req_electricity_out")
    
    # surcharge 
    surcharge = Output(Resources.boolean, value=False)
    @update(state=normal, target=surcharge)
    def update_surcharge_normal(self, dt):
        return False
    
    @update(state=no_power, target=surcharge)
    def update_surcharge_no_power(self, dt):
        return True
    
    
    @transition(source=normal, target=no_power)
    def to_no_power(self):
        return self.electricity_in.value < self.req_sum.value

    @transition(source=no_power, target=normal)
    def to_normal(self):
        return self.electricity_in.value >= self.req_sum.value

    def connect_devices(self, devices):
        for dev in devices:
            api.add(self, f"elec_{dev._name}", Output(Resources.electricity, value=0))
            api.add(self, f"req_{dev._name}", Input(Resources.electricity, value=0))
            
            dev_after_init = getattr(self._parent, dev._name)
            api.add(self._parent, f"connect_elec_{self._name}_to_{dev._name}", Influence(source=f"{self._name}.elec_{dev._name}",
                                                                                               target=dev_after_init.electricity_in))
            api.add(self._parent, f"connect_{dev._name}_to_req_{self._name}", Influence(source=dev_after_init.req_electricity_out,
                                                                                              target=f"{self._name}.req_{dev._name}"))
            api.add(self, f"update_{dev._name}_elec_normal", Update(state=self.normal, target=f"elec_{dev._name}",
                                                                          function=f"(lambda self, dt: self.req_{dev._name}.value)"))
            api.add(self, f"update_{dev._name}_elec_nopower", Update(state=self.no_power, target=f"elec_{dev._name}",
                                                                           function=f"(lambda self, dt: 0)"))
        
        if len(devices) > 0:
            sum_of_reqs_src = " + ".join([f"self.req_{dev._name}.value" for dev in devices])
            update_req_src = f"(lambda self, dt: {sum_of_reqs_src} )"
            self.update_req = Update(state=self.normal, target=self.req_sum, function=update_req_src)
            self.update_req_nopower = Update(state=self.no_power, target=self.req_sum, function=update_req_src)


#elk.plot(Splitter())

In [20]:
# As the splitter do, the WaterSplitter handle the water consumption, from the boiler to the DW and the Shower

@nodependencies
class WaterSplitter(Entity, WaterDevice):
    
    normal = current = State()
    no_water = State()
    
    
    req_wsum = Local(Resources.water, value=0)
    req_wsum_to_req = Influence(source=req_wsum, target="req_water_out")
    
    # surcharge 
    overconsommation = Output(Resources.boolean, value=False)
    @update(state=normal, target=overconsommation)
    def update_overconsommation_normal(self, dt):
        return False
    
    @update(state=no_water, target=overconsommation)
    def update_overconsommation_no_power(self, dt):
        return True
    
    
    @transition(source=normal, target=no_water)
    def to_no_water(self):
        return self.water_in.value < self.req_wsum.value

    @transition(source=no_water, target=normal)
    def to_normal(self):
        return self.water_in.value >= self.req_wsum.value

    def connect_water_devices(self, devices):
        for dev in devices:
            api.add(self, f"water_{dev._name}", Output(Resources.water, value=0)) 
            api.add(self, f"req_{dev._name}", Input(Resources.water, value=0))
            
            dev_after_init = getattr(self._parent, dev._name)
            api.add(self._parent, f"connect_water_{self._name}_to_{dev._name}", Influence(source=f"{self._name}.water_{dev._name}",
                                                                                               target=dev_after_init.water_in))
            api.add(self._parent, f"connect_{dev._name}_to_req_{self._name}", Influence(source=dev_after_init.req_water_out,
                                                                                              target=f"{self._name}.req_{dev._name}"))
            api.add(self, f"update_{dev._name}_water_normal", Update(state=self.normal, target=f"water_{dev._name}",
                                                                          function=f"(lambda self, dt: self.req_{dev._name}.value)"))
            api.add(self, f"update_{dev._name}_water_nowater", Update(state=self.no_water, target=f"water_{dev._name}",
                                                                           function=f"(lambda self, dt: 0)"))
        
        if len(devices) > 0:
            wsum_of_reqs_src = " + ".join([f"self.req_{dev._name}.value" for dev in devices])
            update_req_src = f"(lambda self, dt: {wsum_of_reqs_src} )"
            self.update_req = Update(state=self.normal, target=self.req_wsum, function=update_req_src)
            self.update_req_nowater = Update(state=self.no_water, target=self.req_wsum, function=update_req_src)


elk.plot(WaterSplitter())


In [21]:
# Use by the boiler to get water

class Water_Grid(Entity):

    """ - - - - - - - PORTS - - - - - - - - - - """

    water_In = Input(Resources.water, 20)
    
    water_Need = Output(Resources.water, 0)
    
    water_lack = Local(Resources.water, 0)
    
    
    
    
    """ - - - - - - - STATES & TRANSITIONS - - - - - - - - - - """
    
    state = current = State()
    
    """ - - - - - - - UPDATES - - - - - - - - - - """
    #Update of water_Need=water_lack = cons_Shower+cons_DW
    #1st idea : Water Splitter
    #2nd idea : Dependencies with Shower and DishWasher

In [22]:
# Shower model that consum water when switched on, consumption is adaptable

class Shower(Entity, WaterDevice):

    """ - - - - - - - PORTS - - - - - - - - - - """
    switch_shower = Input(Resources.switch, "off")
        
    water_Need = Local(Resources.water, 8)    
    
    def __init__(self, consumption=8):
        self.water_Need.value = consumption
        
    """ - - - - - - - STATES & TRANSITIONS - - - - - - - - - - """
    
    state = current = State()
    
    """ - - - - - - - UPDATES - - - - - - - - - - """
    
    @update(state=state, target="req_water_out")
    def set_water_out(self,dt):
        if (self.switch_shower.value=="on"):
            return self.water_Need.value
        else:
            return 0

elk.plot(Shower())

In [25]:
# Vacuum robot that can wash a house in 2 times (need to recharge between the two), can work only if no presence is detected 

@dependency(source="req_electricity_out", target="presence_In")
class Vacuum_Robot(Entity, ElectricalDevice):

    """ - - - - - - - PORTS - - - - - - - - - - """
    presence_In = Input(resource=Resources.presence, value="no presence")
    
    charge = Local(resource=Resources.pourcent, value = 70)
    cleaning_progress = Local(resource=Resources.pourcent, value = 0)
    cons_vacuum = Local(Resources.electricity, value = 200)
    
    def __init__(self, consumption=200):
        self.cons_vacuum.value = consumption
    
    """ - - - - - - - STATES & TRANSITIONS - - - - - - - - - - """
    
    off = current = State()
    recharge = State()
    cleaning = State()
    
    off_to_recharge = Transition(source=off, target=recharge, guard=(lambda self: self.charge.value < 100 and
                                                                     self.electricity_in.value >= self.cons_vacuum.value))
    
    recharge_to_cleaning = Transition(source=recharge, target=cleaning, guard=(lambda self: self.charge.value == 100 and
                                                                               self.presence_In.value == "no presence" and
                                                                               self.cleaning_progress.value < 100))
    
    cleaning_to_recharge = Transition(source=cleaning, target=recharge, guard=(lambda self: self.charge.value <= 0 or
                                                                               self.cleaning_progress.value >= 100))
    
    recharge_to_off = Transition(source=recharge, target=off, guard=(lambda self: self.charge.value == 100 and
                                                                     (self.presence_In.value == "detected" or
                                                                      self.cleaning_progress.value >= 100)))
    
    off_to_cleaning = Transition(source=off, target=cleaning, guard=(lambda self: self.electricity_in.value >= self.cons_vacuum.value and
                                                                     self.charge.value >= 100 and self.presence_In.value == "no presence"))
    
    to_off = Transition(source = [recharge, cleaning], target = off, guard=(lambda self: self.electricity_in.value < self.cons_vacuum.value))
    
    
    """ - - - - - - - UPDATES - - - - - - - - - - """
   

    @update(state=recharge, target=charge)
    def set_charge_recharge(self, dt):
        return self.charge.value+5*dt
    
    @update(state=cleaning, target=charge)
    def set_charge_cleaning(self, dt):
        return self.charge.value-2*dt
    
    @update(state=recharge, target="req_electricity_out")
    def set_req_electricity_out_recharge(self,dt):
        return self.cons_vacuum.value
    
    @update(state=off, target="req_electricity_out")
    def set_req_electricity_out_off(self,dt):
        return 0
    
    @update(state=cleaning, target="req_electricity_out")
    def set_req_electricity_out_cleaning(self,dt):
        return 0
    
    @update(state=cleaning, target=cleaning_progress)
    def set__cleaning_progress_cleaning(self,dt):
        return self.cleaning_progress.value+dt
    
elk.plot(Vacuum_Robot())    

In [24]:
# DW can work if the TV is on, it consum water AND electricity

@dependency(source="req_electricity_out", target="switch_usageDW")
class Dishwasher(Entity, ElectricalDevice, WaterDevice):

    """ - - - - - - - PORTS - - - - - - - - - - """
    switch_NoiseDW = Input(Resources.switch, "off")
    switch_usageDW = Input(Resources.switch, "off")
        
    max_Cap = Local(Resources.water, 12)
    actual_Cap = Local(Resources.water, 0)
    
    washing_progress = Local(Resources.pourcent, 0)
    
    cons_DW = Local(Resources.electricity, 900)
        
    def __init__(self, consumption=4000, dw_Capacity=12):
        self.cons_DW.value = consumption
        self.max_Cap.value = dw_Capacity
            
    """ - - - - - - - STATES & TRANSITIONS - - - - - - - - - - """
    
    cleaning = State()
    filling = State()
    off = current = State()
    
    off_to_filling = Transition(source = off, target = filling, guard=(lambda self: self.switch_NoiseDW.value == "off" and self.switch_usageDW.value == "on" ))
    
    filling_to_cleaning = Transition(source = filling, target = cleaning, guard=(lambda self: self.actual_Cap.value >= self.max_Cap.value))
    
    cleaning_to_off  = Transition(source = cleaning, target = off, guard=(lambda self: self.washing_progress.value >= 100))
    
    """ - - - - - - - UPDATES - - - - - - - - - - """
    
    @update(state=[off, filling], target=washing_progress)
    def set_washing_progress_nonCleaning(self,dt):
        return 0
    
    @update(state=cleaning, target=washing_progress)
    def set_washing_progress_cleaning(self,dt):
        return self.washing_progress.value + 2*dt
    
    @update(state=[off, cleaning], target="req_water_out")
    def set_water_out_nonFilling(self,dt):
        return 0
    
    @update(state=filling, target="req_water_out")
    def set_water_out_filling(self,dt):
        return self.max_Cap.value-self.actual_Cap.value
    
    @update(state=filling, target=actual_Cap)
    def set_actual_Cap_filling(self,dt):
        return min(self.actual_Cap.value + 10*dt,self.max_Cap.value)
    
    @update(state=cleaning, target=actual_Cap)
    def set_actual_Cap_cleaning(self,dt):
        return self.actual_Cap.value-0.24
    
    @update(state=off, target=actual_Cap)
    def set_actual_Cap_off(self,dt):
        return 0
    
    @update(state=[cleaning, filling], target="req_electricity_out")
    def set_req_electricity_out_on(self,dt):
        return self.cons_DW.value
    
    @update(state=off, target="req_electricity_out")
    def set_req_electricity_out_off(self,dt):
        return 0
    
#elk.plot(Dishwasher())

dw = Dishwasher()
dw.switch_usageDW.value="on"
simDW = Simulator(dw)
simDW.stabilize()
simDW.advance(52)
simDW.plot()